# COVID-19 data reader

Questo notebook carica i dati regionali riguardanti la diffusione e la gestione del COVID-19 direttamente dal repository delle Protezione civile.

I dati sono così organizzati:
- per ciascuna feature si crea un dizionario i cui elementi sono le regioni, accessibili per nome (es. `trend[indicatore][regione]`)
- ogni elemento contiene il valore misurato nei giorni di osservazione
- le date sono nella lista di stringhe `giorni`
- è stata creata una entry per ciascun dizionario dell'indicatore per l'*Italia*. Questo indicatore, per quanto riguarda gli indicatori di base originali, ne è la somma, sltrimenti è opportunamente calcolato.


Fonte dati: *Presidenza del Consiglio dei Ministri - Dipartimento della Protezione Civile*

https://github.com/pcm-dpc/COVID-19

> **NOTA** Questo notebook nasce per condividere il proprio kernel con altri notebooks dove vengono effettuate e visualizzate le analisi vere e proprie.

## Lista indicatori raccolti nella struttura dati

**Originali:**
- ricoverati_con_sintomi
- terapia_intensiva
- totale_ospedalizzati
- isolamento_domiciliare
- totale_positivi
- variazione_totale_positivi
- nuovi_positivi
- dimessi_guariti
- deceduti
- totale_casi
- tamponi

**Derivati:**
- tamponi_giornalieri
- deceduti_giornalieri
- perc_tamponi_positivi
- perc_guariti_su_positivi
- perc_deceduti_su_positivi

**Altre variabili:**
- giorni
- day_counter
- regioni
- eventi (dizionario di eventi notevoli con data)
    - data (str)
    - id_giorno (indice del giorno dell'evento)
    - desc (str)

In [1]:
import pandas as pd
import numpy as np
import datetime



import warnings
warnings.filterwarnings('ignore')

## Parametri

In [2]:
#data_file='dpc-covid19-ita-regioni.csv' # locale non aggiornato
data_file='https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

## Lettura dati e preprocessing

In [3]:
data=pd.read_csv(data_file)
print(data.columns)

giorni=pd.unique(data['data'])
day_counter=list(range(len(giorni)))
print('Giorni osservati: %d'%len(giorni))
print('Da: %s'%giorni[0][0:10])
print(' A: %s'%giorni[-1][0:10])
regioni=pd.unique(data['denominazione_regione'])

#data[data['denominazione_regione']=='Lombardia'].head(5)
data[data['denominazione_regione']=='Toscana'].tail(5)


Index(['data', 'stato', 'codice_regione', 'denominazione_regione', 'lat',
       'long', 'ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'casi_da_sospetto_diagnostico', 'casi_da_screening',
       'totale_casi', 'tamponi', 'casi_testati', 'note'],
      dtype='object')
Giorni osservati: 149
Da: 2020-02-24
 A: 2020-07-21


,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
3041,2020-07-17T17:00:00,ITA,9,Toscana,43.769231,11.255889,12,1,13,305,...,3,6,8909,1129,10130.0,226.0,10356,384826,257616.0,NaN
3062,2020-07-18T17:00:00,ITA,9,Toscana,43.769231,11.255889,12,1,13,301,...,-4,2,8915,1129,10132.0,226.0,10358,387885,259442.0,NaN
3083,2020-07-19T17:00:00,ITA,9,Toscana,43.769231,11.255889,12,0,12,313,...,11,16,8920,1129,10147.0,227.0,10374,390676,261174.0,NaN
3104,2020-07-20T17:00:00,ITA,9,Toscana,43.769231,11.255889,10,0,10,314,...,-1,1,8921,1130,10148.0,227.0,10375,391998,261958.0,NaN
3125,2020-07-21T17:00:00,ITA,9,Toscana,43.769231,11.255889,11,1,12,314,...,2,9,8927,1131,10157.0,227.0,10384,395061,263474.0,NaN


### Lettura dati popolazione

In [4]:
dati_pop=pd.read_excel('popolazione_regioni.xlsx')
#dati_pop
pop_keys=list(dati_pop['Regione'])


popolazione = {pop_keys[i]: dati_pop.iloc[i]['Popolazione'] for i in range(len(pop_keys))} 

popolazione

{'Lombardia': 10060574,
 'Lazio': 5879082,
 'Campania': 5801692,
 'Sicilia': 4999891,
 'Veneto': 4905854,
 'Emilia-Romagna': 4459477,
 'Piemonte': 4356406,
 'Puglia': 4029053,
 'Toscana': 3729641,
 'Calabria': 1947131,
 'Sardegna': 1639591,
 'Liguria': 1550640,
 'Marche': 1525271,
 'Abruzzo': 1311580,
 'Friuli Venezia Giulia': 1215220,
 'P.A. Trento': 1072276,
 'Umbria': 882015,
 'Basilicata': 562869,
 'Molise': 305617,
 "Valle d'Aosta": 125666,
 'P.A. Bolzano': 106441}

### Inserimento di eventi notevoli riferiti a date

In [5]:
lockdown={'data':'2020-03-10',
          'id_giorno':15,
          'desc':'Lockdown'}

scuole_chiuse={'data':'2020-03-04',
               'id_giorno':9,
               'desc':'Scuole chiuse'}

fase_2={'data':'2020-05-04',
               'id_giorno':71,
               'desc':'Fase2'}


eventi=[scuole_chiuse,lockdown, fase_2]

## Analisi

### Creazione della struttura dati

- Si crea un dizionario per ciascuno degli indicatori selezionati
- a sua volta ciascun elemento del primo dizionario è un dizionario con le regioni come chiave
    - ogni elemento indicatore contiene i valori dell'indicatore per la regione

**Esempio:**
```    
trend[indicatore][regione]
```

In [6]:
# indicatori osservati
indicatori=['ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi']

# qui immetto tutti i trend per ciascuna regione
trend={}
for ind in indicatori:
    trend[ind]={}
    for regione in regioni:
        trend[ind][regione]=np.array(data[data['denominazione_regione']==regione][ind])

# calcolo gli indicatori nazionali come somma delle regioni
for ind in indicatori:
    trend[ind]['Italia']=np.zeros(len(giorni))
    for regione in regioni:
        trend[ind]['Italia']+=trend[ind][regione]

### Indicatori derivati

#### Giorno della settimana associato ai giorni

In [7]:
def which_weekday(datestring):
    # datestring is in this format '2020-02-24T18:00:00'
    return datetime.date(int(datestring[0:4]),int(datestring[5:7]),int(datestring[8:10])).weekday()

giorno_settimana=[which_weekday(d) for d in giorni]

#### Tamponi giornalieri

In [8]:
# tamponi giornalieri
trend['tamponi_giornalieri']={}
for regione in regioni:
    trend['tamponi_giornalieri'][regione]=np.insert(np.diff(trend['tamponi'][regione]),0,0)

trend['tamponi_giornalieri']['Italia']=np.insert(np.diff(trend['tamponi']['Italia']),0,0)


#### Deceduti giornalieri

In [9]:
# deceduti giornalieri
trend['deceduti_giornalieri']={}
for regione in regioni:
    trend['deceduti_giornalieri'][regione]=np.insert(np.diff(trend['deceduti'][regione]),0,0)

trend['deceduti_giornalieri']['Italia']=np.insert(np.diff(trend['deceduti']['Italia']),0,0)


#### Guariti giornalieri

In [10]:
trend['guariti_giornalieri']={}
for regione in regioni:
    trend['guariti_giornalieri'][regione]=np.insert(np.diff(trend['dimessi_guariti'][regione]),0,0)

trend['guariti_giornalieri']['Italia']=np.insert(np.diff(trend['dimessi_guariti']['Italia']),0,0)


#### Percentuale tamponi positivi

In [11]:
# percentuale tamponi positivi su quelli effettuati
trend['perc_tamponi_positivi']={}
for regione in regioni:
    trend['perc_tamponi_positivi'][regione]=100*trend['nuovi_positivi'][regione]/trend['tamponi_giornalieri'][regione]

trend['perc_tamponi_positivi']['Italia']=100*trend['nuovi_positivi']['Italia']/trend['tamponi_giornalieri']['Italia']

#### Rate deceduti e rate guariti su attualmente positivi (il giorno precedente)

In [12]:
trend['perc_guariti_su_positivi']={}
trend['perc_deceduti_su_positivi']={}

np.hstack((regioni,'Italia'))

for regione in np.hstack((regioni,'Italia')):
    # percentuale di GUARITI su POSITIVI
    trend['perc_guariti_su_positivi'][regione]=np.zeros_like(trend['totale_positivi'][regione],dtype=float)
    for d in range(1,len(giorni)):
        try:
            trend['perc_guariti_su_positivi'][regione][d]=100*float(trend['guariti_giornalieri'][regione][d]/trend['totale_positivi'][regione][d-1])
            #print('%d   %4.2f  %4.2f'%(d,trend['guariti_giornalieri'][regione][d],trend['totale_positivi'][regione][d-1]))
        except ValueError:
            trend['perc_guariti_su_positivi'][regione][d]=np.NaN
            
    # percentuale di DECEDUTI su POSITIVI
    trend['perc_deceduti_su_positivi'][regione]=np.zeros_like(trend['totale_positivi'][regione],dtype=float)
    for d in range(1,len(giorni)):
        try:
            trend['perc_deceduti_su_positivi'][regione][d]=100*float(trend['deceduti_giornalieri'][regione][d]/trend['totale_positivi'][regione][d-1])
            #print('%d   %4.2f  %4.2f'%(d,trend['deceduti_giornalieri'][regione][d],trend['totale_positivi'][regione][d-1]))
        except ValueError:
            trend['perc_deceduti_su_positivi'][regione][d]=np.NaN
                

## Salvataggio dati su file PICKLE

Vengono salvati su file **pickle** nell' ordine:
- trend
- regioni
- giorni


In [13]:
import pickle

output_data_file='stored_data.pkl'

outfile = open(output_data_file,'wb')
pickle.dump(trend,outfile)
pickle.dump(regioni,outfile)
pickle.dump(giorni,outfile)
pickle.dump(giorno_settimana,outfile)
pickle.dump(popolazione,outfile)
outfile.close()